In [1]:
import pandas as pd
import numpy as np

In [2]:
charterdf = pd.read_pickle('../../nowdata/charters_2015.pkl')

In [3]:
list(charterdf.columns)

['NCESSCH',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'CMO_WEBTEXT',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'LEAID',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'LEVEL',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'CHARTER_TEXT',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'WEBSITE',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AM',
 'AMALM',
 'AMALF',
 'AS',
 'ASALM',
 'ASALF',
 'HI',
 'HIALM',
 'HIALF',
 'BL',
 'BLALM',
 'BLALF',
 'WH',


In [4]:
range(len(charterdf))

range(0, 12877)

In [14]:
charterdf["WEBTEXT"][0]

[('http://www.maef.net/our-impact/',
  'False',
  '1',
  'Menu\nHome\nAbout Us\nOur Work\nNews/Events\nSupport Us\nContact\nDashboard\nHome\nAbout Us\nOur Work\nNews/Events\nSupport Us\nContact\nDashboard\nOur Impact\nLearn more about how MAEF:\nSupports MCPSS in \ntransforming schools\n across the county to increase high school graduation rates\nIs creating partnerships with \nindustry leaders\n to develop students’ workforce skills in Mobile County Career Academies\nImproves the quality of life in Mobile County by increasing the \nvolunteer base\n and the number of citizens actively in community efforts to improve outcomes for children schools involved citizen engagement across the county Community impact\nIs creating a \ncivic culture\n of continual improvement across Mobile County\nSee below where MAEF programs are making a difference in our public schools.\nGet Directions\nFor:\nDriving\nWalking\nBicycling\nshow options\nhide options\n Avoid Tolls \n Avoid Highways\r              

In [5]:
[len(charterdf["WEBTEXT"][x]) > 0 for x in range(len(charterdf))]

[True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False

In [102]:
#charterdf = charterdf[charterdf["WEBTEXT"].apply(len) > 0] # Drop rows where NO WEBTEXT (0 or 1 words) was gathered (i.e., scrape failed
charterdf = charterdf[charterdf["WEBTEXT"].apply(lambda school: len(school[0][3])) > 0] # Drop rows where first page is empty
charterdf = charterdf[charterdf["WEBTEXT"].apply(lambda school: sum([len(page[3]) for page in school])) > 0] # Drop rows where all pages are empty



IndexError: list index out of range

## Filtering Data 

Determine rows based on if school is not scrapped yet and still open.

Not scrapped: `WEBTEXT` length == 0  
School opened: look at latest SY_STATUS [`SY_STATUS15`]  
    `open:` SY_STATUS15 == 1,3,4,5 or 8  
    `closed:` SY_STATUS15 == 2,6 or 7

In [98]:
filtered_WEBTEXT = (charterdf['WEBTEXT'].str.len() < 2) 

no_urls = charterdf.loc[filtered_WEBTEXT]
#no_urls[['SY_STATUS','WEBTEXT']].head()

In [91]:
filtered_STATUS = (no_urls['SY_STATUS15'] == 1) | (no_urls['SY_STATUS15'] == 3) | (no_urls['SY_STATUS15'] == 4) | (no_urls['SY_STATUS15'] == 5) | (no_urls['SY_STATUS15'] == 8)

filtered_STATUS
charter_df = no_urls.loc[filtered_STATUS]
charter_df.head()

,NCESSCH,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,...,SCH_FTESECURITY_GUA,SCH_FTESERVICES_NUR,SCH_FTESERVICES_PSY,SCH_FTESERVICES_SOC,DISCIPLINE_COUNT,INQUIRY_COUNT,DISC_STR,INQU_STR,NUMWORDS,NUMPAGES


In [66]:
df_URLs = pd.read_pickle("../../scrapy_cluster_data/new_processed_df.pkl")  # Holds URL for each school in 'URL' column
df_URLs = df_URLs[["NCESSCH", "URL"]]

In [56]:
len(df_URLs)

7182

In [57]:
df_URLs["NCESSCH"] = df_URLs["NCESSCH"].astype(float)# Convert to float type (just in case)
charter_df["NCESSCH"] = charter_df["NCESSCH"].astype(float) # Trim down to just the variables we want: unique ID and URL


In [58]:
charter_df = pd.merge(charter_df, df_URLs, how="outer", on="NCESSCH") # Add 'URL' column from df_URLs to charter_df, joining on 'NCESSCH'. how='outer' means keep all keys and rows from both DFs, in this case resulting in a full list of all charter schools and URLs

In [59]:
charter_df.head() # Just double-checking the view

,URL_x,NCESSCH,URL_y,URL
0,NaN,1.301230e+11,http://099.clayton.k12.ga.us/,http://099.clayton.k12.ga.us/
1,NaN,1.301230e+11,http://198.clayton.k12.ga.us/,http://198.clayton.k12.ga.us/
2,NaN,3.500060e+11,http://21stcenturypa.com/wp/,http://21stcenturypa.com/wp/
3,NaN,2.200193e+11,http://42charterschool.org/,http://42charterschool.org/
4,NaN,4.111670e+11,http://a3school.org/,http://a3school.org/


In [45]:
charter_df

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,SCH_FTESERVICES_SOC,DISCIPLINE_COUNT,INQUIRY_COUNT,DISC_STR,INQU_STR,NUMWORDS,NUMPAGES,URL_x,NCESSCH,URL_y


In [47]:
charter_df = charter_df[["NCESSCH", "URL_y"]] # The only variables we need for web-crawling
charter_df = charter_df.drop_duplicates(subset="NCESSCH", keep="first") # Getting rid of duplicate NCESSCH columns (shouldn't be looking over the same school twice)

In [48]:
charter_df.head()

,NCESSCH,URL_y


In [49]:
charter_df = charter_df.drop_duplicates(subset="NCESSCH", keep="first") 

In [50]:
charter_df.columns = ['NCESSCH', 'URL']
charter_df.head() # View of the data

,NCESSCH,URL


In [ ]:
# Prianka's attempt to convert the dataframe to a CSV.
charter_df.to_csv('../data/charter-urls-data-2015.csv') 
# It looks like the conversion happened fine, just not sure where the file outputted to.

End result: save the resulting data frame as a CSV named charter_URLs_2016_unscraped.csv in web_scraping/data and in scrapy-cluster/kafka-monitor/